# Connecting Amazon VPCs
An Amazon VPC is by default a range of private IP addresses, and it requires an array of components to be able to communicate with the public internet. There are various other methods of connecting Amazon VPCs to external networks and services, each providing different features and benefits. It's important to know about these alternative connectivity options so that you can make the most informed decisions when building and maintaining VPCs and their surrounding infrastructure.

This notebook will explore the most commonly used methods for connecting a VPC to another VPC, different AWS services, or an on-premises network. These are as follows:

- *VPC Peering* is for connecting together two VPCs

- *VPC Endpoints* are for connecting your VPC to other public AWS services such as S3 or DynamoDB
- A *Site-to-Site VPN* connection is a simple way to connect an on-premises network to a VPC over the public internet
- *AWS Direct Connect* allows you to connect an on-premises network to a VPC using a private, dedicated connection
- *AWS Transit Gateway* enables multiple VPCs to connect to eachother, and use a Site-to-Site VPN connection or Direct Connect to link an on-premises network

## VPC Peering 
>*VPC peering* is a way of connecting two VPCs that then allows you to route traffic between them using the private IP addresses defined in each VPC. 

### Key Features and Benefits of VPC Peering
- Instances in each VPC can communicate with eachother as if they are in the same VPC

- The connection uses existing AWS infrastructure which means it does not rely on a separate piece of physical hardware or require a separate AWS service. This also means there is no single point of failure or a bandwidth bottleneck.
- You can peer accross AWS accounts, as well as different AWS regions
- You can reference Security Groups in a peered VPC as if they are in your own VPC

### Requirements of VPC Peering
- There must not be overlapping CIDR blocks in each of the VPCs

- VPC peering is not *transitive*, which means that each connection links two single VPCs only. For example, if there is a peering connection between VPCs A and B, and also a peering connection between VPCs B and C, then VPCs A and C are not then automatically peered. You need to set up a separate peering connection between VPCs A and C.
- In order for instances in each VPC to communicate with eachother, you need to configure the route tables in the relevant subnets of each VPC to contain the correct routes

<p align="center"> <img src="images/vpc-peering.png" height="388" width="467"/> </p>

## VPC Endpoints
>*VPC Endpoints* enable you to link your VPC to other AWS services without needing to go through the public internet. 

Without a VPC Endpoint if you want to connect instances in your VPC to another AWS service it requires a relatively complicated set up: an Internet Gateway for public subnets and a NAT Gateway that routes through the Internet Gateway for private subnets. VPC Endpoints provide a one-step solution to allow instances in private or public subnets to connect to other AWS services. They are also redundant, highly-available and horizontally scalable.

There are two types of VPC Endpoints:

### Interface Endpoints
- *Interface Endpoints* have a private IP address provisioned to it by *AWS PrivateLink*, which is a service that uses Network Load Balancers to connect to external AWS services using a high-speed connection

- They are made up of one or more *Elastic Network Interfaces (ENIs)*, which are standalone networking components that allow you to control networking to a compute instance, but keep it separate from the instance
- This then serves as a single entry point to allow traffic to the AWS service, but must be set up with a Security Group to allow traffic out of the VPC
- Interface Endpoints support most AWS services (listed in the image below)
- They are also a paid service charged per hour and per GB of data processed

### Gateway Endpoints:

- *Gateway Endpoints* are designed to be used only with Amazon S3 and DynamoDB (another database service)

- They work by targeting specific routes in a VPC route table to allow traffic to only the supported services 
- They are a free service

<p align="center"> <img src="images/vpc-endpoints.png" height="531" width="604"/> </p>

## Site-to-Site VPN
>A *Site-to-Site VPN (Virtual Private Network)* is one method of establishing a connection between an Amazon VPC and a remote, on-premises network. 

It works by using a VPN connection over the public internet. A VPN creates an encrypted 'tunnel' connection between two devices which ensures that data can be safely and anonymously transmitted over the public internet.

These are the components of a Site-to-Site VPN connection:

- A ***Virtual Private Gateway (VGW)*** is used on the Amazon side to allow for the VPN connection. You create a virtual private gateway in the VPC and associate it with the resources which need access to the Site-to-Site VPN connection.

- A ***Customer Gateway (CGW)*** is needed at the on-premises side. It requires a *Customer Gateway Device*, which is a physical device or software application installed and capable of accepting traffic from the Site-to-Site VPN connection.

<p align=center> <img src=images/S2S_virtual_private_gateway.png width=600 height=300> </p>

## AWS Direct Connect
>*AWS Direct Connect* provides a dedicated, private connection between a remote network and a VPC. 

While a Site-to-Site VPN connection works over the public internet, AWS Direct Connect uses a fibre-optic ethernet connection to directly link together a router in the on-premises infrastructure (a Customer Gateway), with a VPC or other public AWS services such as S3. This is also known as a *Virtual Interface* (*VIF*). 

There are three different types of VIFs that can be configured:

- **Public**: Enables connectivity between an on-premises network and AWS public services, VPCs and Transit Gateways

- **Transit**: Enables private connectivity between an on-premises network and AWS Transit Gateways
- **Private**: Enables connectivity between an on-premises network and an Amazon VPC only. This scenario is illustrated in the image below.

<p align="center"> <img src="images/direct-connect.png" height="484" width="911"/> </p>

## Transit Gateway
>A *Transit Gateway* is an AWS service that acts as a hub between many VPCs, otherwise known as the *hub-and-spoke* connectivity model. 

The problem it solves is that since VPC peering is a non-transitive connection, using peering to connect many VPCs together becomes an unreasonable option at scale (for example, to connect 20 VPCs requires 190 different peering connections). 

The way it works is by acting as a virtual router to route traffic between the VPCs and on-premises networks. The route tables of each VPC connected to the Transit Gateway must be configured properly to limit which VPCs can communicate with eachother.

There are various benefits of an AWS transit gateway:

- It is a highly-available service and can elastically scale depending on the volume of traffic

- To connect an on-premises network to the Transit Gateway you can use either a Site-to-Site VPN connection or a AWS Direct Connect gateway for a dedicated private connection
- It is a regional resource, but you can peer Transit Gateways together across regions, like in the image below

<p align="center"> <img src="images/transit-gateway.png" height="574" width="710"/> </p>

## Key Takeaways
- VPC Peering allows you to connect two separate VPCs as if they were using the same CIDR range, meaning subnets and Security Groups can be used across both VPCs. 
However, it is not a transitive connection.

- VPC Endpoints create a private connection between a VPC and other AWS services. There are two types of VPC Endpoints:
    - Interface Endpoints, which are one or more ENIs using AWS PrivateLink to connect almost any AWS service, but at a financial cost

    - Gateway Endpoints are specific routes in a route table that allow traffic to only Amazon S3 or Dynamo DB, but are free to use

- If you want to connect a VPC to an on-premises network there are two main options:
    - Site-to-Site VPN uses an encrypted connection over the public internet to connect a VGW on the VPC side and a CGW at the on-premises side.

    - AWS Direct Connect uses a VIF on a private, wired connection for high-speed and secure connectivity. The VIF can be public, private or transit.
- A Transit Gateway is a hub-and-spoke model service which enables the connectivity of multiple VPCs with eachother, or with an on-premises network via Site-to-Site VPN or Direct Connect. 